In [9]:
import gradio as gr
from PIL import Image
import random
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

example_images = {
    "플랭크": "plank.jpg",
    "팔굽혀펴기": "pushup.jpg"
}

youtube_videos = {
    "플랭크": [
        "https://www.youtube.com/embed/1T6P56q1FtY",
        "https://www.youtube.com/embed/Zq8nRY9P_cM"
    ],
    "팔굽혀펴기": [
        "https://www.youtube.com/embed/IODxDxX7oi4",
        "https://www.youtube.com/embed/0pkjOk0EiAk"
    ]
}

def process(exercise, image1, image2):
    youtube_url = random.choice(youtube_videos.get(exercise, []))
    youtube_html = f'<iframe width="560" height="315" src="{youtube_url}" frameborder="0" allowfullscreen></iframe>'

    # Load example image
    example_image_path = example_images.get(exercise)
    example_image = cv2.imread(example_image_path, cv2.IMREAD_GRAYSCALE)
    example_image = cv2.resize(example_image, (500, 500))

    # Load uploaded image
    uploaded_image = Image.fromarray(image1)
    uploaded_image = np.array(uploaded_image.convert('L'))
    uploaded_image = cv2.resize(uploaded_image, (500, 500))

    # Compute SSIM between the two images
    score, _ = ssim(example_image, uploaded_image, full=True)

    return youtube_html, f"유사도 점수: {score:.2f}"

def show_example_image(exercise):
    example_image_path = example_images.get(exercise, None)
    if exercise == "팔굽혀펴기":
        return gr.update(visible=True), gr.update(visible=True), example_image_path, "왼쪽 이미지를 보고 올바른 자세로 사진을 찍어주세요."
    elif exercise == "플랭크":
        return gr.update(visible=True), gr.update(visible=False), example_image_path, "왼쪽 이미지를 보고 올바른 자세로 사진을 찍어주세요."
    else:
        return gr.update(visible=False), gr.update(visible=False), None, ""

with gr.Blocks() as blocks:
    with gr.Row():
        exercise_input = gr.Radio(label="어떤 운동인가요?", choices=["플랭크", "팔굽혀펴기"])
        
    with gr.Row():
        example_image_output = gr.Image()
        example_description = gr.HTML()
    
    with gr.Row():
        image_input1 = gr.Image(label="이미지를 업로드해주세요.", visible=False)
        image_input2 = gr.Image(label="이미지를 업로드해주세요.", visible=False)
    
    with gr.Row():
        submit_button = gr.Button("Submit")
        
    with gr.Row():
        youtube_output = gr.HTML(label="유튜브 동영상")
        similarity_score = gr.HTML(label="유사도 점수")

    exercise_input.change(
        fn=show_example_image,
        inputs=exercise_input,
        outputs=[image_input1, image_input2, example_image_output, example_description]
    )

    submit_button.click(
        fn=process,
        inputs=[exercise_input, image_input1, image_input2],
        outputs=[youtube_output, similarity_score]
    )

blocks.launch()


ModuleNotFoundError: No module named 'skimage'